# Show Lines of sights of all bolometry diagnostics in LHD

This notebook describes how to show the lines of sights of all bolometry diagnostics including resistive bolometers and IRVB installed in LHD.

In [ ]:
from pathlib import Path

import numpy as np
from plotly import graph_objects as go
from raysect.core import Ray, World
from raysect.primitive import Mesh
from scipy.spatial.transform import Rotation

from cherab.lhd.machine import __path__, load_pfc_mesh
from cherab.lhd.observer.bolometer import load_irvb, load_resistive
from cherab.lhd.tools.visualization import RMAX, ZMAX

RSM_PATH = Path(__path__[0]).resolve() / "geometry" / "data" / "RSMfiles"

## Load bolometers and show modeled IRVB

Firstly, we load the bolometers into a world scene-graph.

In [ ]:
# World added in scene graph
world = World()

Let us put loaded bolometers into a list.

In [ ]:
cameras = []

In [ ]:
for port, flange in zip(["6.5U", "6.5L"], ["CC01_04", "BC02"], strict=True):
    irvb = load_irvb(port=port, flange=flange, parent=world)
    cameras.append(irvb)

In [ ]:
for port, model in zip(["6.5L", "6.5L", "8O"], ["I", "O", ""], strict=True):
    rb = load_resistive(port=port, model_variant=model, parent=world)
    cameras.append(rb)

Loaded bolometry diagnostics:

In [ ]:
print(cameras)

Let us show the modeled IRVB in 3-D space with plotly.

In [ ]:
irvb = cameras[0]
fig = irvb.plot_bolometer_geometry(plot_pixel_rays={"pixel": (10, 10), "num_rays": 10})
fig.update_layout(
    width=700,
    height=600,
)
fig.show()

## Show bolometry system in 3-D space with plasma

Here we show the bolometry system in 3-D space by plotting the lines of sight of resistive bolometers and view cones of IRVBs.
Plasma volume and some first walls are also shown to see the relative position of the bolometers.

In [ ]:
# Load LHD PFCs and plasma meshes
meshes = load_pfc_mesh(world)
plasma = Mesh.from_file(RSM_PATH / "plasma.rsm", parent=world, name="plasma")

In [ ]:
# instantiate figure object
fig = go.Figure()

# === add bolometers ===============================================================================
for camera, color in zip(cameras, ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"], strict=True):
    # === IRVB LoS =================================================================================
    if "IRVB" in camera.name:
        los = camera.sightline_rays
        # for los_column in los:
        #     for ray in los_column:
        #         line = np.array(
        #             [[*ray.origin],
        #              [*ray.point_on(20)]]
        #         )
        #         fig.add_trace(
        #             go.Scatter3d(
        #                 mode="lines",
        #                 x=line[:, 0],
        #                 y=line[:, 1],
        #                 z=line[:, 2],
        #                 line=dict(color=color, width=2),
        #                 text=camera.name,
        #                 showlegend=False,
        #             )
        #         )
        # === IRVB FoV tetrahedron =====================================================================
        px, py = los.shape
        lines = []
        for i, j in [(0, 0), (px - 1, 0), (px - 1, py - 1), (0, py - 1)]:
            ray = los[i][j]
            lines.append(np.array([[*ray.origin], [*ray.point_on(20)]]))
        slit_centre = camera.slit.centre_point
        for k, is_showlegend in enumerate([False, True]):
            xyz = np.asanyarray(
                [[*[line[k, i] for line in lines], slit_centre[i]] for i in range(3)]
            )
            fig.add_trace(
                go.Mesh3d(
                    x=xyz[0, :],
                    y=xyz[1, :],
                    z=xyz[2, :],
                    i=[1, 1, 0, 0, 3, 1],
                    j=[3, 2, 4, 3, 2, 4],
                    k=[0, 3, 1, 4, 4, 2],
                    color=color,
                    opacity=1.0,
                    flatshading=True,
                    name=camera.name,
                    showlegend=is_showlegend,
                )
            )

    # === RB LoS ===================================================================================
    elif "RB" in camera.name:
        is_showlegend = True
        for foil in camera.foil_detectors:
            ray = Ray(foil.centre_point, foil.sightline_vector)
            line = np.array([[*ray.origin], [*ray.point_on(25)]])

            fig.add_trace(
                go.Scatter3d(
                    mode="lines",
                    x=line[:, 0],
                    y=line[:, 1],
                    z=line[:, 2],
                    line=dict(color=color, width=5),
                    name=camera.name,
                    text=foil.name,
                    showlegend=is_showlegend,
                )
            )
            is_showlegend = False

# === Load Plasma =============================================================
x, y, z = plasma.data.vertices.T
index_i, index_j, index_k = plasma.data.triangles.T

# lighting_effects = dict(ambient=0.4, diffuse=0.5, roughness=0.9, specular=0.6, fresnel=0.2)
plasma_mesh = go.Mesh3d(
    x=x,
    y=y,
    z=z,
    i=index_i,
    j=index_j,
    k=index_k,
    flatshading=False,
    color="#FF00FF",
    opacity=0.3,
    name="Plasma",
    showscale=False,
    hoverinfo="skip",
    showlegend=True,
)

fig.add_trace(plasma_mesh)

# === show LHD 1st wall =======================================================
for pfc_index in [1, 2]:
    mesh = meshes["plates"][pfc_index]

    lighting_effects = dict(
        ambient=0.18, diffuse=1, fresnel=0.1, specular=0, roughness=0.1, facenormalsepsilon=0
    )

    t = mesh.to_root()
    r = Rotation.from_matrix([[t[i, j] for j in range(3)] for i in range(3)])

    x, y, z = r.apply(mesh.data.vertices).T
    index_i, index_j, index_k = mesh.data.triangles.T

    pfc_mesh = go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=index_i,
        j=index_j,
        k=index_k,
        colorscale=[[0, "#e5dee5"], [1, "#e5dee5"]],
        opacity=1.0,
        flatshading=True,
        intensity=z,
        name="First Wall",
        showscale=False,
        hoverinfo="skip",
        lighting=lighting_effects,
        lightposition=dict(x=3000, y=3000, z=10000),
        showlegend=True,
    )
    fig.add_trace(pfc_mesh)

# === XYZ Axis ===================================================================
xaxis_vector = go.Scatter3d(
    x=[0, RMAX * 1.1],
    y=[0, 0],
    z=[0, 0],
    name="X Axis",
    marker=dict(color="rgb(256, 0, 0)", size=2),
    line=dict(color="rgb(256, 0, 0)"),
    showlegend=False,
)
yaxis_vector = go.Scatter3d(
    x=[0, 0],
    y=[0, RMAX * 1.1],
    z=[0, 0],
    name="Y Axis",
    marker=dict(color="rgb(0, 256, 0)", size=2),
    line=dict(color="rgb(0, 256, 0)"),
    showlegend=False,
)
zaxis_vector = go.Scatter3d(
    x=[0, 0],
    y=[0, 0],
    z=[0, ZMAX * 1.5],
    name="Z Axis",
    marker=dict(color="rgb(0, 0, 256)", size=2),
    line=dict(color="rgb(0, 0, 256)"),
    showlegend=False,
)
fig.add_trace(xaxis_vector)
fig.add_trace(yaxis_vector)
fig.add_trace(zaxis_vector)

# === update visibility ==========================================================
fig.update_scenes(
    xaxis_visible=False,
    yaxis_visible=False,
    zaxis_visible=False,
)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=0.7, y=0.7, z=0.7))
fig.update_layout(
    width=700,
    height=600,
    scene_camera=camera,
    scene_aspectmode="data",
    margin=dict(r=10, l=10, b=10, t=10),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)

## Compare flange variant of IRVB 6.5L

Let us compare the flange variant of IRVB 6.5L with the default one (BC02 vs AL01).

In [ ]:
irvbs = []
for port, flange in zip(["6.5L", "6.5L"], ["BC02", "AL01"], strict=True):
    irvb = load_irvb(port=port, flange=flange, parent=world)
    irvbs.append(irvb)

In [ ]:
fig = go.Figure()

# === add bolometers ===============================================================================
for camera, color in zip(irvbs, ["#1f77b4", "#ff7f0e"], strict=True):
    # === IRVB FoV tetrahedron =====================================================================
    if "IRVB" in camera.name:
        los = camera.sightline_rays
        px, py = los.shape
        lines = []
        for i, j in [(0, 0), (px - 1, 0), (px - 1, py - 1), (0, py - 1)]:
            ray = los[i][j]
            lines.append(np.array([[*ray.origin], [*ray.point_on(20)]]))
        slit_centre = camera.slit.centre_point
        for k, is_showlegend in enumerate([True, False]):
            xyz = np.asanyarray(
                [[*[line[k, i] for line in lines], slit_centre[i]] for i in range(3)]
            )
            fig.add_trace(
                go.Mesh3d(
                    x=xyz[0, :],
                    y=xyz[1, :],
                    z=xyz[2, :],
                    i=[1, 1, 0, 0, 3, 1],
                    j=[3, 2, 4, 3, 2, 4],
                    k=[0, 3, 1, 4, 4, 2],
                    color=color,
                    opacity=1.0,
                    flatshading=True,
                    text=camera.name,
                    name=camera.name,
                    showlegend=is_showlegend,
                )
            )

# === Load Plasma =============================================================
x, y, z = plasma.data.vertices.T
index_i, index_j, index_k = plasma.data.triangles.T

# lighting_effects = dict(ambient=0.4, diffuse=0.5, roughness=0.9, specular=0.6, fresnel=0.2)
plasma_mesh = go.Mesh3d(
    x=x,
    y=y,
    z=z,
    i=index_i,
    j=index_j,
    k=index_k,
    flatshading=False,
    color="#FF00FF",
    opacity=0.3,
    name="Plasma",
    showscale=False,
    hoverinfo="skip",
    showlegend=True,
)

fig.add_trace(plasma_mesh)

# === XYZ Axis ===================================================================
xaxis_vector = go.Scatter3d(
    x=[0, RMAX * 1.1],
    y=[0, 0],
    z=[0, 0],
    name="X Axis",
    marker=dict(color="rgb(256, 0, 0)", size=2),
    line=dict(color="rgb(256, 0, 0)"),
    showlegend=False,
)
yaxis_vector = go.Scatter3d(
    x=[0, 0],
    y=[0, RMAX * 1.1],
    z=[0, 0],
    name="Y Axis",
    marker=dict(color="rgb(0, 256, 0)", size=2),
    line=dict(color="rgb(0, 256, 0)"),
    showlegend=False,
)
zaxis_vector = go.Scatter3d(
    x=[0, 0],
    y=[0, 0],
    z=[0, ZMAX * 1.5],
    name="Z Axis",
    marker=dict(color="rgb(0, 0, 256)", size=2),
    line=dict(color="rgb(0, 0, 256)"),
    showlegend=False,
)
fig.add_trace(xaxis_vector)
fig.add_trace(yaxis_vector)
fig.add_trace(zaxis_vector)

# === update visibility ==========================================================
fig.update_scenes(
    xaxis_visible=False,
    yaxis_visible=False,
    zaxis_visible=False,
)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=0.7, y=0.7, z=0.7))
fig.update_layout(
    width=700,
    height=600,
    scene_camera=camera,
    scene_aspectmode="data",
    margin=dict(r=10, l=10, b=10, t=10),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)